In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
def data_process(d_dir, split_rate):
    
    num_class = len(d_dir.keys())

    out = {}
    

    for i in range(num_class):
        x_train, x_test = train_test_split(d_dir[i], test_size=split_rate)
        
        if i == 0:
            train_temp = x_train
            test_temp = x_test
        else:
            train_temp.extend(x_train)
            test_temp.extend(x_test)
    
    random.shuffle(train_temp)
    out['train'] = train_temp
    random.shuffle(test_temp)
    out['test'] = test_temp
    
    return out

In [4]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [5]:
image_dir1 = '/dataset/'
class_name1 = ['untorn', 'torn']

data_dir1 = {}
for i in range(len(class_name1)):
    temp_dir = image_dir1 + class_name1[i]
    img_files = [[os.path.join(temp_dir, f), np.int16(i)] for f in os.listdir(temp_dir) if f.endswith('.png')]
    data_dir1[i] = img_files
print(len(data_dir1[0]), len(data_dir1[1]))

1780 1485


In [6]:
tr_data_dir = data_process(data_dir1, 0.2)

In [7]:
tr_data = np.zeros((len(tr_data_dir['train']), total_pix), dtype=np.float32)
tr_label = np.zeros((len(tr_data_dir['train']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['train'])):
    img = cv2.imread(tr_data_dir['train'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    tr_data[i,:] = img2.flatten()
    tr_label[i] = tr_data_dir['train'][i][1]

In [8]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [9]:
def cnn_model(x, n_class, dropout, reuse, is_training, width=512, height=512):
    with tf.variable_scope('ConvNet', reuse=reuse):
        # RGB data, 3-channel
        input_layer = tf.reshape(x, shape=[-1, height, width, 3])
        
        k_init = tf.contrib.layers.xavier_initializer_conv2d()
        
        conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=48,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv1')
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='pool1')
        # height/2, width/2
        
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv2')
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='pool2')
        # height/4, width/4
        
        conv3 = tf.layers.conv2d(
            inputs=pool2,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv3')
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2, name='pool3')
        # height/8, width/8
        
        conv4 = tf.layers.conv2d(
            inputs=pool3,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv4')
        pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2, name='pool4')
        # height/16, width/16
        
        conv5 = tf.layers.conv2d(
            inputs=pool4,
            filters=96,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu,
            kernel_initializer=k_init,
            name='conv5')
        pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2, name='pool5')
        # height/32, width/32
        
        #pool5_flat = tf.reshape(pool5, [-1, int(height/32) * int(width/32) * 96])
        pool5_flat = tf.contrib.layers.flatten(pool5)
        
        dense1 = tf.layers.dense(pool5_flat, 1024, activation=tf.nn.relu, name='dense1')
        
        dropout = tf.layers.dropout(dense1, rate=dropout, training=is_training, name='drop')
        
        # Output
        out = tf.layers.dense(dropout, n_class, name='output')
        out = tf.nn.softmax(out) if not is_training else out
        
        return out

In [10]:
dropout = 0.5
learning_rate = 0.0001
batch_size = 16
disp_step = 20
n_classes = 2
epochs = 5

tr_steps = int(tr_data.shape[0]/batch_size*epochs)
inner_steps = int((tr_data.shape[0])/batch_size)

In [11]:
with tf.device('/cpu:0'):
    reuse_vars = False
    
    # tf graph input
    X = tf.placeholder(tf.float32, [None, total_pix])
    Y = tf.placeholder(tf.int32, [None, 1])
    
    with tf.device('/gpu:0'):
        # cnn_model(x, n_class, dropout, re_use, is_training, width, height)
        logits_train = cnn_model(X, n_classes, dropout, reuse=reuse_vars,
                                 is_training=True)
        
        logits_test = cnn_model(X, n_classes, dropout, reuse=True,
                                is_training=False)
        
        loss = tf.losses.sparse_softmax_cross_entropy(logits=logits_train, labels=Y)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        grads = optimizer.compute_gradients(loss)
        
        pred = tf.argmax(input=logits_test, axis=1)
        correct_pred = tf.equal(pred, tf.cast(Y, tf.int64))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        #accuracy = tf.metrics.accuracy(labels=Y, predictions=pred)
        
        
        prob = tf.nn.softmax(logits_test, name='softmax_tensor')
        
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        
        reuse_vars = True
        
    
    
    
    
    
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()

In [12]:
    with tf.Session() as sess:

        sess.run(init_g)
        sess.run(init_l)

        t_count = 1

        for j in range(epochs):

            for i in range(inner_steps):

                tr_batch = {}

                if i < inner_steps-1:
                    tr_batch['data'] = tr_data[i*batch_size:(i+1)*batch_size][:]
                    tr_batch['label'] = tr_label[i*batch_size:(i+1)*batch_size][:]

                else:
                    tr_batch['data'] = tr_data[i*batch_size:][:]
                    tr_batch['label'] = tr_label[i*batch_size:][:]

                sess.run(train_op, feed_dict={X: tr_batch['data'],
                                              Y: tr_batch['label']})

                if t_count % disp_step == 1:

                    loss_tr = sess.run(loss, feed_dict = {X: tr_batch['data'],
                                                          Y: tr_batch['label']})

                    acc_tr = sess.run(accuracy, feed_dict = {X: tr_batch['data'],
                                                             Y: tr_batch['label']})
                    
                    prob_tr = sess.run(prob, feed_dict = {X: tr_batch['data'],
                                                         Y: tr_batch['label']})

                    #print('step %d, training accuracy %f, loss %f' % (t_count, accuracy, loss))
                    print('Training step: ', t_count)
                    print('Accuracy', acc_tr)
                    print('Probability')
                    print(prob_tr)
                    print('Loss: ', loss_tr)
                    #print(acc_tr, loss_tr)

                t_count = t_count+1
                
        print("Training is finished!!!")
        
        
        
        ev_accuracy = np.zeros((len(ev_label), 1))
        for k in range(len(ev_label)):
            ev_batch = {}
            ev_batch['data'] = ev_data[k:(k+1)][:]
            ev_batch['label'] = ev_label[k:(k+1)]
            ev_accuracy[k] = sess.run(accuracy, feed_dict={X: ev_batch['data'],
                                                           Y: ev_batch['label']})

        print(np.mean(ev_accuracy))

Training step:  1
Accuracy 0.5
Probability
[[0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]
 [0.26894143 0.7310586 ]]
Loss:  53.688515
Training step:  21
Accuracy 0.53125
Probability
[[0.72621703 0.27378297]
 [0.66441834 0.33558163]
 [0.7106208  0.28937915]
 [0.4076682  0.5923318 ]
 [0.68533206 0.31466797]
 [0.69842887 0.3015711 ]
 [0.71035135 0.28964862]
 [0.7229917  0.27700827]
 [0.67859054 0.3214095 ]
 [0.69019514 0.30980483]
 [0.34521937 0.6547806 ]
 [0.68719023 0.3128098 ]
 [0.36507532 0.63492465]
 [0.65847063 0.3415294 ]
 [0.7166268  0.28337315]
 [0.31595594 0.68404406]]
Loss:  0.57018375
Training step:  41
Accuracy 0.5703125
Probability
[[0.27984264 0.7201574

Training step:  361
Accuracy 0.6875
Probability
[[0.6540063  0.3459937 ]
 [0.6616036  0.33839637]
 [0.6991241  0.30087593]
 [0.65663964 0.34336036]
 [0.7051637  0.29483625]
 [0.5513267  0.44867325]
 [0.6995444  0.30045554]
 [0.56977713 0.43022284]
 [0.66239756 0.33760244]
 [0.52074486 0.47925517]
 [0.5655663  0.43443376]
 [0.68755734 0.31244266]
 [0.35016787 0.6498322 ]
 [0.5175235  0.48247644]
 [0.7073374  0.29266268]
 [0.32472068 0.6752793 ]]
Loss:  0.30691022
Training step:  381
Accuracy 0.515625
Probability
[[0.3535873  0.6464127 ]
 [0.54079175 0.45920828]
 [0.6063953  0.39360476]
 [0.2879648  0.71203524]
 [0.3554785  0.6445215 ]
 [0.33514714 0.66485286]
 [0.6832461  0.31675398]
 [0.6219948  0.3780052 ]
 [0.6001454  0.3998546 ]
 [0.49636176 0.5036382 ]
 [0.6758617  0.32413828]
 [0.50615335 0.49384668]
 [0.35890687 0.64109313]
 [0.54440373 0.45559624]
 [0.6846084  0.31539163]
 [0.7079997  0.29200026]]
Loss:  0.39373147
Training step:  401
Accuracy 0.546875
Probability
[[0.37083843 0

Training step:  721
Accuracy 0.5
Probability
[[0.6874111  0.31258884]
 [0.7242325  0.27576753]
 [0.7220016  0.2779984 ]
 [0.7309398  0.2690602 ]
 [0.7026421  0.29735792]
 [0.7300443  0.26995566]
 [0.72444606 0.2755539 ]
 [0.37599137 0.62400866]
 [0.37382182 0.6261782 ]
 [0.4680465  0.5319535 ]
 [0.26954964 0.73045033]
 [0.27485403 0.725146  ]
 [0.7276473  0.27235267]
 [0.26962525 0.73037475]
 [0.365383   0.634617  ]
 [0.44924203 0.550758  ]]
Loss:  0.30127555
Training step:  741
Accuracy 0.5
Probability
[[0.72996247 0.27003756]
 [0.7308902  0.26910985]
 [0.7199667  0.2800333 ]
 [0.3611288  0.6388712 ]
 [0.72972983 0.27027017]
 [0.33640924 0.6635908 ]
 [0.7307273  0.26927269]
 [0.73010397 0.26989603]
 [0.7101337  0.28986633]
 [0.26965618 0.7303438 ]
 [0.7309977  0.2690023 ]
 [0.26896334 0.73103666]
 [0.41504857 0.5849514 ]
 [0.36806595 0.631934  ]
 [0.26901573 0.73098433]
 [0.30639145 0.6936085 ]]
Loss:  0.10500051
Training step:  761
Accuracy 0.5
Probability
[[0.5658619  0.4341381 ]
 [

85.4517
94.7932